In [ ]:
### function to split documents with Word+Tokenize ###

def split_by_words(text, chunk_size=80, overlap=20):
    words = word_tokenize(text)
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = words[i:i + chunk_size]
        if chunk:
            chunks.append(' '.join(chunk))
    return chunks

# 2. Split only new files into chunks of words
if not new_files:
    print("No new files to process.")
else:
    all_chunks = []
    all_metadatas = []
    for fname in new_files:
        with open(os.path.join(folder_path, fname), 'r', encoding='utf-8') as f:
            text = f.read()
            display(f"Text file: {fname}", Markdown(f"```\n{text}\n```"))
        chunks = split_by_words(text)
        all_chunks.extend(chunks)
        all_metadatas.extend([{"source": fname}] * len(chunks))    

In [ ]:
### get the chunks from a specific file

def get_file_chunks(fname):
    results = vectorstore.get(where={"source": fname})
    file_chunks = results['documents']
    print(f"Found {len(file_chunks)} chunks from {fname}")
    for chunk in file_chunks:
        display(Markdown(f"```\n{chunk}\n```"))

file1_chunks = get_file_chunks("note-1-dt-r.txt")

Found 12 chunks from note-1-dt-r.txt


```
entropy = -p * log2 ( p ) – yeah info gain is difference in entropy before and after split — okay so DT picks feature that max info gain at each node . Gini impurity also similar but faster ? less bias ? not 100 % sure . trees go deep and then prone back ? no , prune . to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right
```

```
. to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right ? sklearn.tree.DecisionTreeClassifier ( max_depth=3 ) — yeah that ’ s what prof used . note : good to visualize trees but not for high dim data . lots of axis-aligned splits , hard to interpret when too many features . oh and trees are unstable — small data change = big model change . they said bagging can help that
```

```
oh and trees are unstable — small data change = big model change . they said bagging can help that — RandomForest . wait then they jumped to regularization — lasso vs ridge . ridge adds λ * sum ( w² ) , shrinks weights , but all stay ≠ 0 . Lasso adds λ * sum ( |w| ) — forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big =
```

```
— forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big = underfit . low λ = overfit . prof wrote this on board : from sklearn.linear_model import Lasso model = Lasso ( alpha=0.1 ) model.fit ( X_train , y_train ) ElasticNet = mix of both ? ratio param controls mix . good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting .
```

...

```
. to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right ? sklearn.tree.DecisionTreeClassifier ( max_depth=3 ) — yeah that ’ s what prof used . note : good to visualize trees but not for high dim data . lots of axis-aligned splits , hard to interpret when too many features . oh and trees are unstable — small data change = big model change . they said bagging can help that
```

```
oh and trees are unstable — small data change = big model change . they said bagging can help that — RandomForest . wait then they jumped to regularization — lasso vs ridge . ridge adds λ * sum ( w² ) , shrinks weights , but all stay ≠ 0 . Lasso adds λ * sum ( |w| ) — forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big =
```

```
— forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big = underfit . low λ = overfit . prof wrote this on board : from sklearn.linear_model import Lasso model = Lasso ( alpha=0.1 ) model.fit ( X_train , y_train ) ElasticNet = mix of both ? ratio param controls mix . good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting .
```

```
good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting . btw they said don ’ t scale trees but do scale for lasso etc . bcz regularization depends on magnitude . std scaling or minmax okay . might try gridsearch to tune alpha — sklearn.model_selection.GridSearchCV decision boundary of tree is step-like , not smooth like linear models . ok Q : why trees overfit more than lasso ? more flexible
```

```
step-like , not smooth like linear models . ok Q : why trees overfit more than lasso ? more flexible model class I think ?
```

- Analysis: The [word_toneknize] splits the punctiation into separate tokens and creates this view that makes harder to compreehend the context. I researched other tokenizer and found a "de-tokenizer" [tree bank word] to join the tokens in a more natural way.

In [ ]:
# clean Chroma DB on terminal:
# rm -rf ./data/chroma_db
# deleted list from "processed_files.txt"

# experiment HDBSCAN:
the results were -1, which means that everything was considered noise. 

In [ ]:
### function to retrieve the chunks from a specific file

def get_file_chunks(fname):
    results = vectorstore.get(where={"source": fname})
    file_chunks = results['documents']
    print(f"Found {len(file_chunks)} chunks from {fname}")
    for chunk in file_chunks:
        display(Markdown(f"```\n{chunk}\n```"))             # <-- display the contents of each chunk for conference
    return file_chunks                                      # <-- return the list for further use
 

In [ ]:
# 4.1. Get embeddings chunks from File 1

file1_chunks = get_file_chunks("note-1-dt-r.txt")
file1_embeddings = embeddings.embed_documents(file1_chunks)


# 4.2. Initialize and fit HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=2)
labels = cluster.fit_predict(file1_embeddings)

print("Cluster labels for File 1:", labels)

Found 6 chunks from note-1-dt-r.txt


```
entropy = -p * log2 (p) – yeah info gain is difference in entropy before and after split — okay so DT picks feature that max info gain at each node . Gini impurity also similar but faster? less bias? not 100% sure . trees go deep and then prone back? no, prune . to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right
```

```
. to prevent overfitting . training error low but generalization bad . CART uses binary splits – only yes/no right? sklearn.tree.DecisionTreeClassifier (max_depth=3) — yeah that ’ s what prof used . note: good to visualize trees but not for high dim data . lots of axis-aligned splits, hard to interpret when too many features . oh and trees are unstable — small data change = big model change . they said bagging can help that
```

```
oh and trees are unstable — small data change = big model change . they said bagging can help that — RandomForest . wait then they jumped to regularization — lasso vs ridge . ridge adds λ * sum (w²), shrinks weights, but all stay ≠ 0 . Lasso adds λ * sum (|w|) — forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big =
```

```
— forces some to zero . ohhh good for feature selection . balance bias-variance tradeoff . λ too big = underfit . low λ = overfit . prof wrote this on board: from sklearn.linear_model import Lasso model = Lasso (alpha=0.1) model.fit (X_train, y_train) ElasticNet = mix of both? ratio param controls mix . good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting.
```

```
good when multicollinearity or many small coeffs . visualize loss function — lasso diamond corners cause zeros . interesting . btw they said don ’ t scale trees but do scale for lasso etc . bcz regularization depends on magnitude . std scaling or minmax okay . might try gridsearch to tune alpha — sklearn.model_selection.GridSearchCV decision boundary of tree is step-like, not smooth like linear models . ok Q: why trees overfit more than lasso? more flexible
```

```
step-like, not smooth like linear models . ok Q: why trees overfit more than lasso? more flexible model class I think?
```

Cluster labels for File 1: [-1 -1 -1 -1 -1 -1]


In [ ]:
file2_chunks = get_file_chunks("note-2-ul.txt")
file2_embeddings = embeddings.embed_documents(file1_chunks)


# 4.2. Initialize and fit HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=2)
labels = cluster.fit_predict(file2_embeddings)

print("Cluster labels for File 2:", labels)

Found 8 chunks from note-2-ul.txt


```
unsup = no labels . kmeans = simplest one but still used a lot . init k centroids randomly (k-means++ better), assign pts, recalc centroids, repeat . converge when no pt changes . but result depends on init + scale . scale important! feature w bigger range dominates dist calc — always standardize first . elbow method = plot inertia vs k — look for bend, but not always obvious . inertia =
```

```
. elbow method = plot inertia vs k — look for bend, but not always obvious . inertia = sum of dist² to centroid . alt metric = silhouette score — between -1 and 1. close to 1 = well-clustered . clustering ≠ classification . labels are not known . use cases: market segmentation, gene expr clustering, anomaly detection (esp dbscan). DBSCAN better for weird shapes, dense clusters — uses eps +
```

```
, anomaly detection (esp dbscan). DBSCAN better for weird shapes, dense clusters — uses eps + min_samples . tricky to tune tho . forms clusters based on density, noisy pts marked as outliers (label -1). sklearn DBSCAN ex: from sklearn.cluster import DBSCAN model = DBSCAN (eps=0.5, min_samples=5) model.fit (X) hierarchical clustering = agglomerative or divisive — we focus on bottom-up (agglomerative). start w
```

```
(X) hierarchical clustering = agglomerative or divisive — we focus on bottom-up (agglomerative). start w all pts as indiv cluster, merge closest pairs step by step . dendrogram = tree of merges . can "cut" tree at diff levels = diff num clusters . linkage: single, complete, avg . sklearn has AgglomerativeClustering . before clustering, can reduce dim (PCA) for speed + viz . tSNE/UMAP for
```

```
sklearn has AgglomerativeClustering . before clustering, can reduce dim (PCA) for speed + viz . tSNE/UMAP for 2D plot = better for human eye but not for modeling . PCA = linear, tSNE = non-linear . tSNE distorts structure globally . good for pattern discovery . spectral clustering = build similarity graph → Laplacian → eigenvectors → k-means in lower-dim eigenspace . nice when structure is graphy, not spherical . most clustering algos rely on
```

```
→ k-means in lower-dim eigenspace . nice when structure is graphy, not spherical . most clustering algos rely on distance metric — Euclidean default . alt: cosine sim (for text), manhattan, etc . eval: hard bcz no true label . silhouette best for most . DB index too . compare within/between cluster distance . can also visualize clusters to judge quality . problem: k-means assumes spherical clusters, equal size . not
```

```
. can also visualize clusters to judge quality . problem: k-means assumes spherical clusters, equal size . not true for real-world . if data has diff density or shapes → fails . open Q: how to know if clusters mean anything in real world? scaling is essential — StandardScaler or MinMaxScaler from sklearn . pipeline ex: from sklearn.pipeline import make_pipeline pipe = make_pipeline (StandardScaler (), KMeans (n_clusters=3)) pipe.fit (
```

```
: from sklearn.pipeline import make_pipeline pipe = make_pipeline (StandardScaler (), KMeans (n_clusters=3)) pipe.fit (X)
```

Cluster labels for File 2: [-1 -1 -1 -1 -1 -1]


In [ ]:
file3_chunks = get_file_chunks("note-3-dt-r-ul.txt")
file3_embeddings = embeddings.embed_documents(file3_chunks)


# 4.2. Initialize and fit HDBSCAN
cluster = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=2)
labels = cluster.fit_predict(file3_embeddings)

print("Cluster labels for File 1:", labels)

Found 7 chunks from note-3-dt-r-ul.txt


```
lecture today was fast af . started with trees . entropy vs gini impurity — diff metrics to decide best split . both OK. CART = binary tree = each node has 2 splits . sklearn uses this . code: from sklearn.tree import DecisionTreeClassifier model = DecisionTreeClassifier (criterion='entropy', max_depth=4) trees prone to overfit — esp if depth unbounded . pruning = way to fix . early stopping or post-prune . bagging helps = ensemble.
```

```
if depth unbounded . pruning = way to fix . early stopping or post-prune . bagging helps = ensemble . RandomForest = multiple trees on bootstrapped samples + rand subset of features per split . reduces variance . trees = interpretability good, but unstable to data changes . then prof jumped into regression . regularization = shrink model capacity . Ridge = L2 norm = λ * Σ (w²). Lasso = L1 norm = λ *
```

```
. Ridge = L2 norm = λ * Σ (w²). Lasso = L1 norm = λ * Σ|w| . Ridge keeps all weights ≠ 0, Lasso can zero out → sparse . Lasso good for feature selection . ElasticNet = mix of both — good if features correlated code ex: from sklearn.linear_model import ElasticNet model = ElasticNet (alpha=0.1, l1_ratio=0.5) model.fit (X, y) important: scale features before fitting regularized
```

```
= ElasticNet (alpha=0.1, l1_ratio=0.5) model.fit (X, y) important: scale features before fitting regularized models — otherwise magnitudes skew the penalty . StandardScaler or RobustScaler if outliers . tune α via cross-val — use GridSearchCV or RandomizedSearchCV . metrics: RMSE, R² . underfit vs overfit — regularization helps balance bias/var tradeoff . last part was unsupervised learning . clustering w/o labels . k-means = most used . init centers, assign pts
```

```
last part was unsupervised learning . clustering w/o labels . k-means = most used . init centers, assign pts, recalc, repeat . problem: sensitive to init . use k-means++ . elbow method not always clear . silhouette score better maybe . PCA + k-means often combined for vis . t-SNE only for viz — not for modeling . clusters in t-SNE are sometimes fake . DBSCAN = cluster via density . can detect noise . great
```

```
modeling . clusters in t-SNE are sometimes fake . DBSCAN = cluster via density . can detect noise . great for shape-agnostic clusters . params hard to tune tho . hierarchical = dendrograms . use ‘ ward ’ linkage . but slow w/ big data . general note: sklearn models consistent API — fit / predict / score . also: why trees don ’ t need scaling? bcz splits based on order not value . contrast w/
```

```
also: why trees don ’ t need scaling? bcz splits based on order not value . contrast w/ reg models . each algo has tradeoffs — no free lunch! choose based on data, task, interpretability needs.
```

Cluster labels for File 1: [-1 -1 -1 -1 -1 -1 -1]
